## Sequence Classification Problem

We will define a simple sequence classification problem to explore bidirectional LSTMs.

The problem is defined as a sequence of random values between 0 and 1. 

A binary label (0 or 1) is associated with each input. Initially, the output values are all 0. Once the cumulative sum of the input values in the sequence exceeds a threshold, then the output value flips from 0 to 1.

A threshold of 1/4 the sequence length is used.

For example, below is a sequence of 10 input timesteps (X):

```python
0.63144003 0.29414551 0.91587952 0.95189228 0.32195638 0.60742236 0.83895793 0.18023048 0.84762691 0.29165514
```

In this case the threshold is `2.5` and the corresponding classification output (y) would be:

```python
0 0 0 1 1 1 1 1 1 1
```

In [1]:
import random as rand
from random import random
import numpy as np
import math
import tensorflow as tf

In [2]:
# create a sequence classification instance
def get_sequence(sequence_length):
    # create a sequence of random numbers in [0,1]
    X = np.array([random() for _ in range(sequence_length)])
    # calculate cut-off value to change class values
    limit = sequence_length / 4.0
    # determine the class outcome for each item in cumulative sequence
    y = np.array([0 if x < limit else 1 for x in np.cumsum(X)])
    
    return X, y

# create n examples with random sequence lengths between 5 and 15
def get_examples(n):
    X_list = []
    y_list = []
    sequence_length_list = []
    for _ in range(n):
        sequence_length = rand.randrange(start=5, stop=15)
        X, y = get_sequence(sequence_length)
        X_list.append(X)
        y_list.append(y)
        sequence_length_list.append(sequence_length)
    
    return X_list, y_list, sequence_length_list

# Tensorflow requires that all sentences (and all labels) inside the same batch have the same length
# so we have to pad the data (and labels) inside the batches with 0's
def pad(sentence, max_length):
    pad_len = max_length - len(sentence)
    padding = np.zeros(pad_len)
    return np.concatenate((sentence, padding))
    
# create batches
def batch(data, labels, sequence_lengths, batch_size=2):
    n_batch = int(math.ceil(len(data) / batch_size))
    index = 0
    for _ in range(n_batch):
        batch_sequence_lengths = np.array(sequence_lengths[index: index + batch_size])
        batch_length = np.array(max(batch_sequence_lengths)) # max length in batch
        batch_data = np.array([pad(x, batch_length) for x in data[index: index + batch_size]]) # pad data
        batch_labels = np.array([pad(x, batch_length) for x in labels[index: index + batch_size]]) # pad labels
        index += batch_size
        
        # reshape input data to be suitable for LSTMs
        batch_data = batch_data.reshape(-1, batch_length, 1)
        
        yield batch_data, batch_labels, batch_length, batch_sequence_lengths

In [3]:
x_train, y_train, sequence_length_train = get_examples(100)
x_test, y_test, sequence_length_test = get_examples(30)
for d, l, max_l, lengths in batch(x_train, y_train, sequence_length_train):
    print(d)
    print(l)
    print(max_l)
    print(lengths)
    break

[[[ 0.73755389]
  [ 0.26961524]
  [ 0.88026586]
  [ 0.9989053 ]
  [ 0.00747029]
  [ 0.72430263]
  [ 0.82384136]
  [ 0.56453923]
  [ 0.65806141]
  [ 0.86632174]
  [ 0.        ]
  [ 0.        ]]

 [[ 0.84252824]
  [ 0.84096689]
  [ 0.72784385]
  [ 0.37255124]
  [ 0.73089072]
  [ 0.05947006]
  [ 0.3028108 ]
  [ 0.51689167]
  [ 0.30126617]
  [ 0.46847721]
  [ 0.80309845]
  [ 0.31077745]]]
[[ 0.  0.  0.  1.  1.  1.  1.  1.  1.  1.  0.  0.]
 [ 0.  0.  0.  0.  1.  1.  1.  1.  1.  1.  1.  1.]]
12
[10 12]


In [4]:
# https://stackoverflow.com/questions/39808336/tensorflow-bidirectional-dynamic-rnn-none-values-error
# https://www.tensorflow.org/api_docs/python/tf/nn/bidirectional_dynamic_rnn
# https://github.com/tensorflow/tensorflow/tree/master/tensorflow/contrib/crf
# https://guillaumegenthial.github.io/sequence-tagging-with-tensorflow.html

In [5]:
# bidirectional lstm + CRF
learning_rate   = 0.001
training_epochs = 100
input_size = 1
batch_size  = 32
num_units = 128 # the number of units in the LSTM cell
number_of_classes = 2

input_data   = tf.placeholder(tf.float32, [None, None, input_size], name="input_data")
labels = tf.placeholder(tf.int32, shape=[None, None], name="labels") # shape = (batch, sentence)
batch_sequence_length = tf.placeholder(tf.int32)
original_sequence_lengths = tf.placeholder(tf.int32, [None])

# scope is mandatory to use LSTMCell
with tf.name_scope("BiLSTM"):
    with tf.variable_scope('forward'):
        lstm_fw_cell = tf.nn.rnn_cell.LSTMCell(num_units, forget_bias=1.0, state_is_tuple=True)
    with tf.variable_scope('backward'):
        lstm_bw_cell = tf.nn.rnn_cell.LSTMCell(num_units, forget_bias=1.0, state_is_tuple=True)
    (output_fw, output_bw), states = tf.nn.bidirectional_dynamic_rnn(cell_fw=lstm_fw_cell, 
                                                                     cell_bw=lstm_bw_cell, 
                                                                     inputs=input_data,
                                                                     sequence_length=original_sequence_lengths, 
                                                                     dtype=tf.float32,
                                                                     scope="BiLSTM")

# As we have Bi-LSTM, we have two output, which are not connected. So merge them
outputs = tf.concat([output_fw, output_bw], axis=2)

# fully connected layer
W = tf.get_variable(name="W", shape=[2 * num_units, number_of_classes],
                dtype=tf.float32)

b = tf.get_variable(name="b", shape=[number_of_classes], dtype=tf.float32,
                initializer=tf.zeros_initializer())

outputs_flat = tf.reshape(outputs, [-1, 2 * num_units])
pred = tf.matmul(outputs_flat, W) + b
scores = tf.reshape(pred, [-1, batch_sequence_length, number_of_classes])

# linear-CRF
log_likelihood, transition_params = tf.contrib.crf.crf_log_likelihood(scores, labels, original_sequence_lengths)

loss = tf.reduce_mean(-log_likelihood)

# Compute the viterbi sequence and score.
viterbi_sequence, viterbi_score = tf.contrib.crf.crf_decode(scores, transition_params, original_sequence_lengths)

# training
optimizer = tf.train.AdamOptimizer(learning_rate)
train_op = optimizer.minimize(loss)

C:\Users\felipe.zschornack\AppData\Local\Continuum\Anaconda2\envs\tensorflow\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [6]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    
    # Train for a fixed number of iterations.
    for i in range(training_epochs):
        for batch_data, batch_labels, batch_seq_len, batch_sequence_lengths in batch(x_train, y_train, sequence_length_train, batch_size):
            tf_viterbi_sequence, _ = session.run([viterbi_sequence, train_op], 
                                                 feed_dict={input_data: batch_data, 
                                                            labels: batch_labels, 
                                                            batch_sequence_length: batch_seq_len,
                                                            original_sequence_lengths: batch_sequence_lengths })
            if i % 10 == 0:
                # mask to correct input sizes
                mask = (np.expand_dims(np.arange(batch_seq_len), axis=0) <
                    np.expand_dims(batch_sequence_lengths, axis=1))
                total_labels = np.sum(batch_sequence_lengths)
                correct_labels = np.sum((batch_labels == tf_viterbi_sequence) * mask)
                accuracy = 100.0 * correct_labels / float(total_labels)
                print("Accuracy: %.2f%%" % accuracy)

Accuracy: 51.33%
Accuracy: 53.62%
Accuracy: 53.97%
Accuracy: 56.25%
Accuracy: 87.33%
Accuracy: 89.13%
Accuracy: 91.43%
Accuracy: 90.62%
Accuracy: 90.67%
Accuracy: 92.03%
Accuracy: 93.33%
Accuracy: 90.62%
Accuracy: 91.67%
Accuracy: 93.48%
Accuracy: 92.06%
Accuracy: 96.88%
Accuracy: 94.33%
Accuracy: 94.20%
Accuracy: 94.92%
Accuracy: 96.88%
Accuracy: 95.33%
Accuracy: 94.57%
Accuracy: 95.56%
Accuracy: 100.00%
Accuracy: 96.00%
Accuracy: 95.65%
Accuracy: 95.87%
Accuracy: 100.00%
Accuracy: 96.33%
Accuracy: 96.38%
Accuracy: 96.51%
Accuracy: 100.00%
Accuracy: 97.33%
Accuracy: 96.74%
Accuracy: 96.83%
Accuracy: 100.00%
Accuracy: 97.00%
Accuracy: 96.74%
Accuracy: 97.46%
Accuracy: 100.00%


## References
1. Tutorial this code was based on: [Bi-LSTM + CRF with character embeddings for NER and POS](https://guillaumegenthial.github.io/sequence-tagging-with-tensorflow.html)
2. Bidirectional LSTM in keras (contains the description of the problem used in this code): [How to Develop a Bidirectional LSTM For Sequence Classification in Python with Keras](https://machinelearningmastery.com/develop-bidirectional-lstm-sequence-classification-python-keras/)
3. [Example of Bidirectional LSTM implementation in Tensorflow](https://stackoverflow.com/questions/39808336/tensorflow-bidirectional-dynamic-rnn-none-values-error)
4. [Example of CRF implementation in Tensorflow](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/contrib/crf)
5. Explains in details the LSTM "num_units" parameter: [Understanding LSTM in Tensorflow](https://jasdeep06.github.io/posts/Understanding-LSTM-in-Tensorflow-MNIST/)
6. Explains how variable sequence lengths work in Tensorflow: [Variable Sequence Lengths in TensorFlow](https://danijar.com/variable-sequence-lengths-in-tensorflow/)